# Integrate Agenta with DSPy

This notebook demonstrates how to connect **Agenta** with **DSPy** for comprehensive observability and debugging of your LLM applications.

> **What is Agenta?** [Agenta](https://agenta.ai) is an open-source LLMOps platform designed to streamline the deployment, management, and scaling of large language models. It offers comprehensive observability, testing, and deployment capabilities for AI applications.

> **What is DSPy?** [DSPy](https://dspy-docs.vercel.app/) ([GitHub](https://github.com/stanfordnlp/dspy)) is a framework for algorithmically optimizing LM prompts and weights. It provides composable and declarative modules for instructing language models in a more systematic way than traditional prompting.

## Implementation Guide

Follow this tutorial to set up DSPy with Agenta's observability platform for real-time application insights.

### Step 1: Install Required Dependencies

Install the necessary Python packages for this integration:

In [ ]:
!pip install agenta dspy openinference-instrumentation-dspy

**Package Descriptions:**
- `agenta`: Core SDK for Agenta's prompt engineering and observability platform
- `dspy`: Framework for building systematic LLM applications with prompt optimization
- `openinference-instrumentation-dspy`: Automatic instrumentation library for DSPy operations

### Step 2: Setup and Configuration

Configure your environment and initialize the Agenta SDK:

In [ ]:
import os
import agenta as ag
import dspy
from openinference.instrumentation.dspy import DSPyInstrumentor


# Load configuration from environment
os.environ["AGENTA_API_KEY"] = "your_agenta_api_key"
os.environ["AGENTA_HOST"] = (
    "https://cloud.agenta.ai"  # Optional, defaults to the Agenta cloud API
)
os.environ["OPENAI_API_KEY"] = "your_openai_api_key"  # Required for OpenAI models


# Start Agenta SDK
ag.init()

**What does `ag.init()` do?**
This function initializes the Agenta SDK and sets up the necessary configuration for observability. It establishes connection to the Agenta platform, configures tracing and logging settings, and prepares the instrumentation context for your application.

### Step 3: Enable DSPy Monitoring

Initialize the OpenInference DSPy instrumentation to automatically capture DSPy operations:

In [ ]:
# Activate DSPy monitoring
DSPyInstrumentor().instrument()

### Step 4: Configure DSPy Language Model

Set up your DSPy language model configuration:

In [ ]:
# Configure DSPy with your preferred language model
lm = dspy.LM("openai/gpt-4o")
dspy.configure(lm=lm)

### Step 5: Build Your Instrumented Application

Here's a complete example showcasing multiple DSPy use cases with Agenta instrumentation:

#### Use Case 1: Math Reasoning with Chain of Thought

In [ ]:
@ag.instrument()
def math_reasoning(question: str):
    cot = dspy.ChainOfThought("question -> answer: float")
    response = cot(question=question)
    return response

#### Use Case 2: Retrieval-Augmented Generation (RAG) with Wikipedia Search

In [ ]:
@ag.instrument(spankind="query")
def search_wikipedia(query: str) -> list[str]:
    results = dspy.ColBERTv2(url="http://20.102.90.50:2017/wiki17_abstracts")(
        query, k=3
    )
    return [x["text"] for x in results]


@ag.instrument()
def rag(question: str):
    cot = dspy.ChainOfThought("context, question -> response")
    response = cot(context=search_wikipedia(question), question=question)
    return response

#### Use Case 3: Article Generation with Outline and Sections

In [ ]:
class Outline(dspy.Signature):
    """Outline a thorough overview of a topic."""
    
    topic: str = dspy.InputField()
    title: str = dspy.OutputField()
    sections: list[str] = dspy.OutputField()
    section_subheadings: dict[str, list[str]] = dspy.OutputField(
        desc="mapping from section headings to subheadings"
    )


class DraftSection(dspy.Signature):
    """Draft a top-level section of an article."""
    
    topic: str = dspy.InputField()
    section_heading: str = dspy.InputField()
    section_subheadings: list[str] = dspy.InputField()
    content: str = dspy.OutputField(desc="markdown-formatted section")


class DraftArticle(dspy.Module):
    def __init__(self):
        self.build_outline = dspy.ChainOfThought(Outline)
        self.draft_section = dspy.ChainOfThought(DraftSection)
    
    def forward(self, topic):
        outline = self.build_outline(topic=topic)
        sections = []
        for heading, subheadings in outline.section_subheadings.items():
            section, subheadings = f"## {heading}", [
                f"### {subheading}" for subheading in subheadings
            ]
            section = self.draft_section(
                topic=outline.title,
                section_heading=section,
                section_subheadings=subheadings,
            )
            sections.append(section.content)
        return dspy.Prediction(title=outline.title, sections=sections)


@ag.instrument()
def journalist(topic: str):
    draft_article = DraftArticle()
    article = draft_article(topic=topic)
    return article

#### Run the Examples

In [ ]:
# Use case 1: Chain of Thought reasoning
response = math_reasoning("What is 2 + 2?")
print("Chain of Thought response:", response)

In [ ]:
# Use case 2: RAG with Wikipedia
rag_response = rag("What's the name of the castle that David Gregory inherited?")
print("RAG response:", rag_response)

In [ ]:
# Use case 3: Article generation
article = journalist("The impact of AI on society")
print("Article generation response:", article)

### Step 6: Understanding the @ag.instrument() Decorator

The `@ag.instrument()` decorator automatically captures all input and output data from your function, enabling comprehensive observability without manual instrumentation.

**Span Type Configuration:**
Use the `spankind` parameter to categorize operations in Agenta WebUI. Available options:

- `agent` - Autonomous agent behaviors
- `chain` - Sequential processing workflows
- `workflow` - Complete application processes (default)
- `tool` - Utility and helper functions
- `embedding` - Vector embedding operations
- `query` - Search and retrieval tasks
- `completion` - Text generation operations
- `chat` - Conversational interfaces
- `rerank` - Result ordering operations

**Standard Behavior:**
By default, when `spankind` is not specified, the operation becomes a root-level span, categorized as a `workflow` in Agenta.

In [ ]:
# Example with custom span classification:
@ag.instrument(spankind="query")
def search_knowledge_base(search_term: str):
    # Knowledge base search implementation
    pass

### Step 7: View Traces in Agenta

After running your application, access detailed execution traces through Agenta's dashboard. The observability data includes:

- Complete workflow execution timeline
- DSPy module initialization and configuration steps
- Chain of Thought reasoning processes
- Retrieval operations and context augmentation
- Language model calls and response generation
- Performance metrics and timing analysis


<img 
    style="display: block; margin: 20px; text-align: center"
    src="./images/agenta-openinference-dspy-trace.png"
    width="90%"
    alt="Agenta dashboard showing DSPy application trace with detailed execution steps">


The observability interface provides insights for:
- Debug complex reasoning chains and prompt optimization
- Monitor retrieval effectiveness and context quality
- Analyze language model performance and token usage
- Track application behavior trends and optimization opportunities

## Next Steps

For more detailed information about Agenta's observability features and advanced configuration options, visit the [Agenta Observability SDK Documentation](/observability/observability-sdk).